<a href="https://colab.research.google.com/github/asliddin2002developer/ML-Projects/blob/main/Tashken_House_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
# df.head()
df['price'].replace('Договорная', df.price.min(), inplace=True)
df.price = df.price.astype(int)
df['size'].replace('Площадьземли:1сот', df['size'].min(), inplace=True)
df['size'] = df['size'].astype(float)

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=34)

housing = train_set.drop('price', axis=1)
housing_labels = train_set['price'].copy()

housing_num = housing.drop(['location', 'district'], axis=1)

##if we need to create our transformer
# from sklearn.base import BaseEstimator, TransformerMixin


In [99]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy='median')),
    ("std_scaler", StandardScaler())
])

num_pipeline.fit_transform(housing_num)

array([[ 1.2793082 ,  0.04953239,  1.48964554,  0.75592793],
       [-0.57604124, -0.03488367,  1.03869842,  1.13982286],
       [-1.50371596, -0.05920695,  0.13680418, -0.77965178],
       ...,
       [-0.57604124, -0.04346836, -0.31414294, -0.39575685],
       [ 0.35163348, -0.02772977, -0.76509006, -0.77965178],
       [-1.50371596, -0.04919148,  0.13680418, -0.39575685]])

In [100]:
from sklearn.compose import ColumnTransformer

num_attrbs = list(housing_num)
cat_attrbs = ['location', 'district']

full_pipeline = ColumnTransformer([
      ('num', num_pipeline, num_attrbs),
      ('cat', OrdinalEncoder(), cat_attrbs)

])


In [101]:
housing_prepared = full_pipeline.fit_transform(housing)

In [34]:
housing_prepared[0]

array([1.27930820e+00, 4.95323886e-02, 1.48964554e+00, 7.55927931e-01,
       1.10400000e+03, 8.00000000e+00])

In [42]:
housing_prepared

array([[ 1.27930820e+00,  4.95323886e-02,  1.48964554e+00,
         7.55927931e-01,  1.10400000e+03,  8.00000000e+00],
       [-5.76041242e-01, -3.48836740e-02,  1.03869842e+00,
         1.13982286e+00,  1.35900000e+03,  1.10000000e+01],
       [-1.50371596e+00, -5.92069462e-02,  1.36804183e-01,
        -7.79651775e-01,  1.89000000e+02,  2.00000000e+00],
       ...,
       [-5.76041242e-01, -4.34683583e-02, -3.14142938e-01,
        -3.95756848e-01,  2.40000000e+01,  1.00000000e+00],
       [ 3.51633478e-01, -2.77297704e-02, -7.65090058e-01,
        -7.79651775e-01,  6.94000000e+02,  6.00000000e+00],
       [-1.50371596e+00, -4.91914812e-02,  1.36804183e-01,
        -3.95756848e-01,  1.06800000e+03,  8.00000000e+00]])

##Linear Regression

In [43]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

LR_model.fit(housing_prepared, housing_labels)

LinearRegression()

In [46]:
test_data = housing.sample(10)
test_data


,location,district,rooms,size,level,max_levels
4736,"город Ташкент, Олмазорский район, Каракамыш кв...",Олмазорский,2,58.0,5,9
2730,"город Ташкент, Мирабадский район, Фидокор (Чех...",Мирабадский,3,70.0,2,4
1817,"город Ташкент, Юнусабадский район, ц-5 Киёт (Ц...",Юнусабадский,2,90.0,2,8
3397,"город Ташкент, Мирзо-Улугбекский район, Буюк И...",Мирзо-Улугбекский,2,40.0,1,3
5387,"город Ташкент, Чиланзарский район, Гулистан",Чиланзарский,3,70.0,4,4
5569,"город Ташкент, Чиланзарский район, Гулистон",Чиланзарский,3,72.0,4,9
6091,"город Ташкент, Мирзо-Улугбекский район, Паркен...",Мирзо-Улугбекский,4,180.0,3,9
2018,"город Ташкент, Шайхантахурский район, Бельтепа",Шайхантахурский,4,125.0,1,5
446,"город Ташкент, Чиланзарский район, Лутфи",Чиланзарский,3,75.0,9,9
6095,"город Ташкент, Мирабадский район, 1-й проезд С...",Мирабадский,3,81.0,3,5


In [49]:
test_labels = housing_labels.loc[test_data.index]
test_labels

4736     29500
2730     66000
1817    110000
3397     26000
5387     40000
5569     51500
6091    110000
2018    118000
446      60000
6095     41500
Name: price, dtype: int64

In [51]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[-5.76041242e-01, -3.63144547e-02,  5.87751303e-01,
         1.13982286e+00,  4.45000000e+02,  3.00000000e+00],
       [ 3.51633478e-01, -2.77297704e-02, -7.65090058e-01,
        -7.79651775e-01,  1.41000000e+02,  1.00000000e+00],
       [-5.76041242e-01, -1.34219632e-02, -7.65090058e-01,
         7.55927931e-01,  1.10500000e+03,  8.00000000e+00],
       [-5.76041242e-01, -4.91914812e-02, -1.21603718e+00,
        -1.16354670e+00,  2.15000000e+02,  2.00000000e+00],
       [ 3.51633478e-01, -2.77297704e-02,  1.36804183e-01,
        -7.79651775e-01,  6.90000000e+02,  6.00000000e+00],
       [ 3.51633478e-01, -2.62989896e-02,  1.36804183e-01,
         1.13982286e+00,  6.91000000e+02,  6.00000000e+00],
       [ 1.27930820e+00,  5.09631693e-02, -3.14142938e-01,
         1.13982286e+00,  3.08000000e+02,  2.00000000e+00],
       [ 1.27930820e+00,  1.16166995e-02, -1.21603718e+00,
        -3.95756848e-01,  8.62000000e+02,  7.00000000e+00],
       [ 3.51633478e-01, -2.41528186e-02,  2.391

In [52]:
predicted_labels = LR_model.predict(test_data_prepared) 
predicted_labels

array([ 41068.34516964,  88843.90170042,  45166.64632218,  67137.20793446,
        89865.55424824, 104041.99057433, 122674.54125494, 133407.30536916,
        72992.24952387, 111456.18478231])

In [53]:
test_labels

4736     29500
2730     66000
1817    110000
3397     26000
5387     40000
5569     51500
6091    110000
2018    118000
446      60000
6095     41500
Name: price, dtype: int64

In [54]:
pd.DataFrame({'Predicted': predicted_labels, 'Original': test_labels})

,Predicted,Original
4736,41068.345170,29500
2730,88843.901700,66000
1817,45166.646322,110000
3397,67137.207934,26000
5387,89865.554248,40000
5569,104041.990574,51500
6091,122674.541255,110000
2018,133407.305369,118000
446,72992.249524,60000
6095,111456.184782,41500


#Predicting the prize using test_set and comparing it with test_labels

In [87]:
X_test = test_set.drop('price',axis=1)
X_test


,location,district,rooms,size,level,max_levels
808,"город Ташкент, Чиланзарский район, Чиланзар-18",Чиланзарский,3,65.00,4,4
4922,"город Ташкент, Учтепинский район, Чиланзар ква...",Учтепинский,2,44.00,4,5
3345,"город Ташкент, Мирзо-Улугбекский район, Буз-1 ...",Мирзо-Улугбекский,2,50.00,5,5
4061,"город Ташкент, Яшнободский район, Паркент",Яшнободский,3,75.00,5,9
3631,"город Ташкент, Чиланзарский район, Algaritm",Чиланзарский,2,63.09,5,9
...,...,...,...,...,...,...
301,"город Ташкент, Юнусабадский район, Кашкар",Юнусабадский,2,46.00,2,4
1842,"город Ташкент, Юнусабадский район, Минор",Юнусабадский,4,84.00,3,4
3767,"город Ташкент, Шайхантахурский район, ц-13 Лаб...",Шайхантахурский,1,42.00,2,4
1026,"город Ташкент, Мирабадский район, Якуба Коласа",Мирабадский,3,80.00,4,4


In [61]:
y_test = test_set['price'].copy()
y_test.astype(float)

808     38500.0
4922    36000.0
3345    29500.0
4061     1000.0
3631    42000.0
         ...   
301     36000.0
1842    73000.0
3767    37000.0
1026    78000.0
1390    48000.0
Name: price, Length: 1513, dtype: float64

In [102]:
X_test_prepared = full_pipeline.fit_transform(X_test)

In [103]:
X_test_prepared

array([[ 3.19446008e-01, -3.64499475e-02,  1.25404363e-01,
        -7.81478927e-01,  3.52000000e+02,  5.00000000e+00],
       [-5.78921669e-01, -4.80084618e-02,  1.25404363e-01,
        -4.03480968e-01,  2.62000000e+02,  4.00000000e+00],
       [-5.78921669e-01, -4.47060291e-02,  5.57606644e-01,
        -4.03480968e-01,  8.80000000e+01,  1.00000000e+00],
       ...,
       [-1.47728935e+00, -4.91092727e-02, -7.39000199e-01,
        -7.81478927e-01,  4.21000000e+02,  6.00000000e+00],
       [ 3.19446008e-01, -2.81938658e-02,  1.25404363e-01,
        -7.81478927e-01,  6.20000000e+01,  0.00000000e+00],
       [ 3.19446008e-01, -3.64499475e-02, -7.39000199e-01,
        -4.03480968e-01,  3.30000000e+02,  5.00000000e+00]])

In [105]:
y_predicted = LR_model.predict(X_test_prepared)

In [106]:
y_predicted

array([131128.00607976,  94906.41403559,  51868.38023053, ...,
        86669.12751063,  68251.72692471, 148896.38038878])

#Mean Absolute Error

In [108]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)

In [110]:
print('MAE=', mae)

MAE= 80803.35516810672


# Root Mean Squared Error

In [119]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 654160.2772923369


#Random Forest

In [116]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(housing_prepared, housing_labels)


RandomForestRegressor()

In [117]:
y_predicted = RF_model.predict(X_test_prepared)

In [123]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 654160.2772923369


#Cross Validation

In [126]:
X = df.drop('price', axis=1)
y = df['price'].copy()

X_prepared = full_pipeline.fit_transform(X)

Cross Validation with Linear Regression

In [139]:
from sklearn.model_selection import cross_val_score

mse_scores = cross_val_score(LR_model, X_prepared, y, scoring='neg_mean_squared_error', cv=25)

In [137]:
def display_scores(scores):
  print('Scores:', scores)
  print('Mean:', scores.mean())
  print("Std.dv:", scores.std())

In [140]:
display_scores(np.sqrt(-mse_scores))

Scores: [  34278.74189343  100124.59362883   42906.7369998    35168.63748939
   36910.39623849  219255.71339879   47200.26384116   35204.69954696
   46167.86420282   38589.75160629   45188.15245544   45559.70869647
   56222.82254099   38730.28862825   43267.64862522   46661.52638958
   60469.5342972    33188.22508083   47104.56999205 2986778.99376562
   81825.25421959  244868.29945251  888379.02706059  380124.90695067
  327339.80526691]
Mean: 238460.64649071492
Std.dv: 589826.6613739843


Cross Validation with Random Forest

In [141]:
RF_mse_scores = cross_val_score(RF_model, X_prepared, y, scoring='neg_mean_squared_error', cv=15)
display_scores(np.sqrt(-RF_mse_scores))

Scores: [  73996.25825692   60103.8533319   601277.83602379  224599.46057259
  210201.55972295   45262.05040168  199774.79297394 1135311.66551008
  104807.79525264  334298.99945135  239957.21185634 2313952.31552114
  208041.23123976  689453.96144779  546284.72280646]
Mean: 465821.58095795446
Std.dv: 571142.0011876965


#pickle 

In [142]:
import pickle

filename = 'LR_model.pkl'
with open(filename, 'wb') as file:
  pickle.dump(LR_model, file)

In [143]:
with open(filename, 'rb') as file:
  model = pickle.load(file)

#joblib

In [146]:
import joblib

filename = 'RF_model.jbl'
joblib.dump(RF_model, filename)

['RF_model.jbl']

In [148]:
rf_model = joblib.load(filename)

In [149]:
RF_mse_scores = cross_val_score(rf_model, X_prepared, y, scoring='neg_mean_squared_error', cv=15)
display_scores(np.sqrt(-RF_mse_scores))

Scores: [  77525.12710586   55272.59331411  578182.44896123  234129.20005956
  302645.94976088   45897.56512396  150791.97628902 1044409.86323315
   50449.86800581  363603.80447985  322198.0679606  2313946.31629593
  195994.20139033  689863.23216758  494487.51504171]
Mean: 461293.1819459713
Std.dv: 563636.5229416031
